IMPORTS AND CONFIGURATION

In [1]:
import numpy as np
import pandas as pd
import os
import importlib
import sys
import copy
import diskcache

In [2]:
backtesting_path = r'C:\Users\vinayak\Desktop\Backtesting'
print("path before", sys.path)
if backtesting_path not in sys.path:
    print("The backtesting folder was not in the systems path")
    sys.path.append(backtesting_path)
print("path after", sys.path)
print()

path before ['c:\\Program Files\\Python312\\python312.zip', 'c:\\Program Files\\Python312\\DLLs', 'c:\\Program Files\\Python312\\Lib', 'c:\\Program Files\\Python312', '', 'C:\\Users\\vinayak\\AppData\\Roaming\\Python\\Python312\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32\\lib', 'c:\\Program Files\\Python312\\Lib\\site-packages\\Pythonwin']
The backtesting folder was not in the systems path
path after ['c:\\Program Files\\Python312\\python312.zip', 'c:\\Program Files\\Python312\\DLLs', 'c:\\Program Files\\Python312\\Lib', 'c:\\Program Files\\Python312', '', 'C:\\Users\\vinayak\\AppData\\Roaming\\Python\\Python312\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32\\lib', 'c:\\Program Files\\Python312\\Lib\\site-packag

In [3]:
# from scipy.optimize import minimize
from Modules import Plot
from Modules import Helpers
from Modules import Data as data
from Modules import Utility as util
from Modules import TradeAndLogics as TL
from Modules import Data_Processing as dp
from Modules.enums import Option, LongShort, DB, FNO, Leg, OHLC, PTSL

In [4]:
pwd()

'c:\\Users\\vinayak\\Desktop\\Backtesting\\Dispersion\\Optimizations\\Intraday'

In [5]:
original_stdout = util.const(sys.stdout)
original_stdout.value

TRADING/ STRATEGY PARAMETERS

In [8]:
entry = '2023-07-12 12:00:00'
exit = '2023-07-13 14:45:00'
ICView = LongShort.Long

In [9]:
entry = pd.to_datetime(entry)
exit = pd.to_datetime(exit)

In [35]:
from datetime import timedelta
start_date = pd.to_datetime(entry - timedelta(1))
end_date = pd.to_datetime(exit + timedelta(1))

In [10]:
index_symbol = 'BANKNIFTY'
expiry_type_near = 'I'
expiry_type_next = 'II'
risk_free_rate = 0.1 # (10% interest rate)
timeframe = 1 # mins
look_back_window = 25*4
# Trade/ Strategy Parameters
buying_delta_threshold_per_lot = 5
selling_delta_threshold_per_lot = 1
notional_vega_buying = 10000 # rs
notional_vega_selling = 8000 # rs
profit_target = 100000
stop_loss = 100000
zscore_threshold_long = 2
zscore_threshold_short = -2
ic_threshold_long = 0.8
ic_threshold_short = 0.2
epsilon = 0.1
moneyness_ironfly = 0
price_factor = 5

In [11]:
def tell_delta_threshold_per_lot(ticker):
    if ticker.intention == LongShort.Long:
        return buying_delta_threshold_per_lot
    return selling_delta_threshold_per_lot

def tell_notional_vega(ticker):
    if ticker.intention == LongShort.Long:
        return notional_vega_buying
    return notional_vega_selling

In [12]:
strategy_desc = f"{entry.strftime('%d%b_%H_%M')}_{exit.strftime('%d%b_%H_%M')}"
strategy_desc

'12Jul_12_00_13Jul_14_45'

Strategy Specific Data Dealing and Modifications

In [13]:
index_symbol = 'BANKNIFTY'
index_lot_size = 15

NEAR MONTH (I) DATA FETCHING AND PROCESSING

In [14]:
import Dispersion.DispersionAdjustedFunctionality as daf

In [15]:
basket = daf.RawWeightedPortfolio()
basket.insert('HDFCBANK', 550, 27.04)
basket.insert('ICICIBANK', 700, 23.03)
basket.insert('KOTAKBANK', 400, 11.72)
basket.insert('SBIN', 750, 11.27)
basket.insert('AXISBANK', 625, 11.18)

In [16]:
logs_near = open(f'LOGS_Fetching_Near_Month_Data_for_{strategy_desc}.txt', 'w')
sys.stdout = logs_near
constituents_near = {}
ohlc = OHLC.close
index_near = daf.ticker(index_symbol, index_lot_size, True, start_date, end_date, expiry_type_near, True, timeframe, True, 0.1)
index_near.initializeDispersion(constituents_near, False, 1, ICView)
index_near.set_ohlc(ohlc)
for stock in basket.Symbols():
    constituents_near[stock] = daf.ticker(stock, basket.LotSize(stock), True, start_date, end_date, expiry_type_near, True, timeframe, False, 0.1)
    constituents_near[stock].initializeDispersion({}, True, basket.Weight(stock), ICView.opposite())
    constituents_near[stock].set_ohlc(ohlc)
sys.stdout = original_stdout.value
logs_near.close()

In [17]:
index = copy.deepcopy(index_near)

In [18]:
def get_vega_legs(ticker, timestamp, *Legs):
    vega = 0
    for leg in Legs:
        if leg.Instrument == FNO.FUTURES:
            continue
        greeks = ticker.greeks(timestamp, leg.Instrument, leg.Strike)
        if greeks is None:
            return None
        vega += greeks['vega'] * leg.Position.value
    return abs(vega)

In [19]:
# def get_lots_for_neutrality(logging_information, vegas, weights, notional_vega):    
#     lots_for_neutrality = []
#     for symbol, _ in vegas.items():
#         lots = notional_vega * weights[symbol] / vegas[symbol]
#         lots_for_neutrality.append(lots)
#         logging_information[symbol]['Lots to take position'] = int(lots)
#     return int(np.round(lots_for_neutrality[0])), np.round(lots_for_neutrality[1:]).astype(int)

In [20]:
def get_proportionality_lots(logging_information, ticker, ticker_value, ticker_weight, notional_scaling_value):
    lots_ticker = int(np.round(notional_scaling_value * ticker_weight/ ticker_value))
    logging_information['Lots to take position'] = lots_ticker
    return lots_ticker

In [21]:
def entry_signal(timestamp):
    if timestamp == entry:
        return True
    return False
def exit_signal(timestamp):
    if timestamp == exit:
        return True
    return False

In [22]:
def get_lots_for_entry(ticker, timestamp, **kwargs):
    logging_information = {}
    legs = Helpers.get_legs_ironfly_price(ticker, timestamp, 1, ticker.intention, moneyness_ironfly, price_factor)
    vega_ticker_ironfly = get_vega_legs(ticker, timestamp, *legs)
    if vega_ticker_ironfly is None:
        return None, None
    unweighted_vega = vega_ticker_ironfly * ticker.lot_size
    notional_vega = tell_notional_vega(ticker)
    idea_for_weight = ticker.weight
    logging_information = {
        'Lot Size': ticker.lot_size,
        'Weight': ticker.weight,
        'Vega per IronFly': vega_ticker_ironfly,
        'Vega per Lot IronFly': vega_ticker_ironfly * ticker.lot_size,
        'Target Notional Vega': notional_vega * ticker.weight
    }
    
    # if 'vega_neutral' in kwargs and kwargs['vega_neutral']:
    
    if 'ic_neutral' in kwargs and kwargs['ic_neutral'] == True:
        if ticker.is_component:
            idea_for_weight *= math.sqrt(ticker.get_ic_at(timestamp))

    elif 'theta_neutral' in kwargs and kwargs['theta_neutral'] == True:
        strike_ticker, _ = ticker.find_moneyness_strike(timestamp, 0, Option.Call)
        logging_information['ATM Strike'] = strike_ticker
        if ticker.is_component:
            import math
            idea_for_weight *= math.sqrt(ticker.get_ic_at(timestamp))

    lots_ticker = get_proportionality_lots(logging_information, ticker, unweighted_vega, idea_for_weight, notional_vega)
    print(f">> {ticker.symbol}")
    for info_key, info_value in logging_information.items():
        print(f"  {info_key}: {info_value}")
    return lots_ticker

In [23]:
def take_dispersion_position(timestamp, remarks, ticker, lots):
    legs = []
    legs = Helpers.get_legs_ironfly_price(ticker, timestamp, lots, ticker.intention, moneyness_ironfly, price_factor)
    for leg in legs:
        key = f'{ticker.symbol}_{leg.id()}'
        if key not in ticker.tokens.keys():
            ticker.tokens[key] = TL.Token(ticker, leg.Instrument, leg.Strike, leg.LegName)
        token = ticker.tokens[key]
        token.add_position(timestamp, lots, leg.Position)
    ticker.take_position(timestamp, remarks, *legs)


In [24]:
class DispersionPTSL(TL.PTSLHandling):
    def __init__(self, profit_target, stop_loss, *tickers):
        super().__init__(profit_target, stop_loss, *tickers)

    def is_valid(self, timestamp):
        # if PTSL is active, update to make sure it's correct
        if self.active_ptsl:
            self.update_validity(timestamp)
        return not self.active_ptsl   
    
    def update_validity(self, timestamp):
        if timestamp.date() != self.triggered_at.date():
            self.reset()
            print(f"New day, PTSL for Intraday Strategy is resetted")
        # index = self.tickers[0]
        # for z in index.df_futures['zscore'].loc[self.triggered_at : timestamp]:
        #     if abs(z) <= epsilon:
        #         self.triggered_at = None
        #         self.active_ptsl = False  
        #         print(f"PTSL reverted back to mean (<= {epsilon}): Future Signals can now be used for trading")

In [25]:
def squareoff(time_ix, timestamp, remarks, logging = True):
    print("************  SQUARE OFF BEGINS  *********** ")
    # print("******  Information updated globally. TICKER WISE UPDATION BEGINS  ******")
    TL.squareoff_ticker(time_ix, timestamp, remarks, index, logging)
    for component in index.components.values():
        TL.squareoff_ticker(time_ix, timestamp, remarks, component, logging)
    print("* SQUARE OFF TRADES SAVED, TOKENS IN TICKERS UPDATED")
    print("************  SQUARE OFF COMPLETE  *********** ")


In [26]:
def isTodayAnyExpiry(timestamp, index):
    is_expiry = False
    is_expiry = is_expiry or index.is_expiry(timestamp)
    for component in index.components.values():
        is_expiry = is_expiry or component.is_expiry(timestamp)
    return is_expiry

In [27]:
def UpdateDispersionTickers(time_ix, timestamp, remarks, hedge, logging_token_update, logging_hedging):
    index_delta_threshold_per_lot = tell_delta_threshold_per_lot(index)
    TL.HandleUpdate(index, time_ix, timestamp, remarks, index_delta_threshold_per_lot, hedge, logging_token_update, logging_hedging)
    for component in index.components.values():
        component_delta_threshold_per_lot = tell_delta_threshold_per_lot(component)
        TL.HandleUpdate(component, time_ix, timestamp, remarks, component_delta_threshold_per_lot, hedge, logging_token_update, logging_hedging)

In [28]:
def zoom_dispersion_trade(start, end, file_name):
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    portfolio = [index, *[component for component in index.components.values()]]
    performance = Helpers.zoom_tokens_performance_bar_by_bar(*portfolio, start=start, end=end)

    # for name, df in performance.items():
    #     if name != "Combined Summary":
    #         # df['Z-Score'] = index.df_futures['zscore'].loc[start:end]
    #         # df['IC'] = index.df_futures['ic'].loc[start:end]

    Plot.save_df_to_excel(performance, file_name)

In [29]:
def visualise_dispersion_trade(start, end, file_name):
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    portfolio = [index, *[component for component in index.components.values()]]
    result_df = Helpers.get_summary_portfolio(*portfolio, start=start, end=end)

    # result_df['Z-Score'] = index.df_futures['zscore'].loc[start:end]
    # result_df['IC'] = index.df_futures['ic'].loc[start:end]

    fig = Plot.plot_df(result_df, *(result_df.columns))
    Plot.save_plot(fig, file_name)

In [30]:
def save_dispersion_trades(start, end, file_name):
    trades_dict = {}
    portfolio = [index, *[comp for comp in index.components.values()]]
    trades_dict['All Trades'] = Helpers.get_trades_portfolio(*portfolio, start=start, end=end)
    for ticker in portfolio:
        trades_dict[ticker.symbol] = Helpers.get_trades_ticker(ticker, start=start, end=end)
    Plot.save_df_to_excel(trades_dict, file_name)

In [32]:
file = open(f'LOGS_Trading_Logic_for_{strategy_desc}.txt', 'w', buffering = 1)
sys.stdout = file

index.reset_trades()
for component in index.components.values():
    component.reset_trades()

TrackPTSL = DispersionPTSL(profit_target, stop_loss, index, *index.components.values())

current_position = None
trade_start_date, trade_end_date, trade_count = None, None, 0
for time_ix, timestamp in enumerate(index.timestamps):
    print()
    print(f"Timestamp: {timestamp}")
    #######################################################################################################################################################
    if TL.check_existing_position(index):
        
        print(f"Existing Position Check: TRUE | Dispersion position: {current_position}")
        
        # 3:20 (Market Close) SQUAREOFF
        if exit_signal(timestamp):
            print("************  MARKET CLOSE  *********** ")
            UpdateDispersionTickers(time_ix, timestamp, 'NoHedging', False, True, False)
            squareoff(time_ix, timestamp, f'3:20/Market Close Square Off', True)
            trade_end_date = timestamp
            trade_count+=1
            zoom_dispersion_trade(trade_start_date, trade_end_date, f"Trade_{strategy_desc}.xlsx")
            visualise_dispersion_trade(trade_start_date, trade_end_date, f"Visualise_Trade_{strategy_desc}.html")
            continue

        # PROFIT TARGET AND STOP LOSS SQUAREOFF
        if TrackPTSL.is_valid(timestamp) and TrackPTSL.status(timestamp) != PTSL.Valid:
            print("************  TrackPTSL Trigger hit *********** ")
            print(f"{TrackPTSL.nature} Square off at NetPnl of {TrackPTSL.pnl_last_trade}")
            UpdateDispersionTickers(time_ix, timestamp, 'NoHedging', False, True, False)
            squareoff(time_ix, timestamp, f'{TrackPTSL.nature} Square Off', True)
            trade_end_date = timestamp
            trade_count+=1
            zoom_dispersion_trade(trade_start_date, trade_end_date, f"Trade_{strategy_desc}.xlsx")
            visualise_dispersion_trade(trade_start_date, trade_end_date, f"Visualise_Trade_{strategy_desc}.html")
            continue
            
        # IF NO NEED TO SQUAREOFF, HEDGE IF NEEDED
        UpdateDispersionTickers(time_ix, timestamp, 'Delta Hedging using Synthetic Futures', True, True, True)
        print("========================================================================================================================================================================")
        continue
    #######################################################################################################################################################
    
    
    #######################################################################################################################################################
    if entry_signal(timestamp):
        print("************  ENTRY TIME REACHED  ************  ")
        UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, False, False)
        if isTodayAnyExpiry(timestamp, index):
            print("Expiry Day today. Trading on Expiry Day is not Allowed")
            continue

        if not TrackPTSL.is_valid(timestamp):
            print(f"{TrackPTSL.nature} was Triggered at {TrackPTSL.triggered_at.strftime('%H:%M on %d/%b/%Y')} and Z-Score has not yet reverted back to mean (abs(z) is not yet <= {epsilon})")
            continue
        
        # try:
        lots_index = get_lots_for_entry(index, timestamp)
        if lots_index == 0:
            print("Cant take Dispersion Position, lots index for target notional vega can't be 0")
            continue
        take_dispersion_position(timestamp, f'{ICView.name} IC', index, lots_index)
        
        for component in index.components.values():
            lots_component = get_lots_for_entry(component, timestamp)
            take_dispersion_position(timestamp, f'{ICView.name} IC', component, lots_component)
        
        TrackPTSL.fresh_trade(timestamp)
        current_position = ICView
        trade_start_date = timestamp
        print(f"{ICView} IC Trade executed")
        # except Exception as e:
        #     print(f"Couldn't Enter a {ICView} IC Trade Position: {e}")  

        continue
    #######################################################################################################################################################

    UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, True, False)
    print("========================================================================================================================================================================")
    

file.close()
sys.stdout = original_stdout.value

In [35]:
portfolio = [index, *[comp for comp in index.components.values()]]

In [37]:
index.Trades.tradesArr

[{'Timestamp': Timestamp('2023-07-12 12:00:00'),
  'Remarks': 'Long IC',
  'Position Leg(0)': <LongShort.Short: -1>,
  'Lots Leg(1)': 22,
  'Instrument Leg(2)': <Option.Put: -1>,
  'Strike Leg(3)': 43600.0,
  'Price Leg(4)': np.float64(85.45),
  'LegName Leg(5)': 'IronFly/FarPut'},
 {'Timestamp': Timestamp('2023-07-13 14:45:00'),
  'Remarks': '3:20/Market Close Square Off',
  'Position Leg(0)': <LongShort.Long: 1>,
  'Lots Leg(1)': 22,
  'Instrument Leg(2)': <Option.Put: -1>,
  'Strike Leg(3)': 43600.0,
  'Price Leg(4)': np.float64(82.4),
  'LegName Leg(5)': 'IronFly/FarPut'}]

In [36]:
Helpers.get_trades_ticker(index, start=trade_start_date, end=trade_end_date)

,Symbol,Remarks,Position Leg(0),Lots Leg(1),Instrument Leg(2),Strike Leg(3),Price Leg(4),LegName Leg(5)
Timestamp,,,,,,,,
2023-07-12 12:00:00,BANKNIFTY,Long IC,LongShort.Short,22,Option.Put,43600.0,85.45,IronFly/FarPut
2023-07-13 14:45:00,BANKNIFTY,3:20/Market Close Square Off,LongShort.Long,22,Option.Put,43600.0,82.40,IronFly/FarPut


In [38]:
hdfc = index.components['HDFCBANK']

In [45]:
hdfc.tokens

{'HDFCBANK_Call_1760.0': <Modules.TradeAndLogics.Token at 0x1afb90b43e0>,
 'HDFCBANK_Call_1650.0': <Modules.TradeAndLogics.Token at 0x1afb9267680>,
 'HDFCBANK_Put_1650.0': <Modules.TradeAndLogics.Token at 0x1afbe8adc10>,
 'HDFCBANK_Put_1570.0': <Modules.TradeAndLogics.Token at 0x1afbe8ad4c0>}

In [49]:
hdfc.lot_size

550

In [50]:
hdfc.tokens['HDFCBANK_Call_1650.0'].data.iloc[s+2]

np.float64(34.5)

In [48]:
hdfc.tokens['HDFCBANK_Call_1650.0'].stats.iloc[s:e]

,instrument,net_lots,position,net_delta,net_vega,underlying_iv,running_pnl_without_expenses,expenses,running_pnl
2023-07-12 11:59:00,Option.Call,0.0,LongShort.Neutral,0.000000,0.000000,0.000000,0.0,0.000,0.000000e+00
2023-07-12 12:00:00,Option.Call,4.0,LongShort.Short,0.000000,0.000000,0.000000,0.0,143.374,-1.433740e+02
2023-07-12 12:01:00,Option.Call,4.0,LongShort.Short,-1161.937541,-2935.070762,0.243978,101420.0,143.374,1.012766e+05
2023-07-12 12:02:00,Option.Call,4.0,LongShort.Short,-1162.979505,-2934.999428,0.244332,205920.0,143.374,2.057766e+05
2023-07-12 12:03:00,Option.Call,4.0,LongShort.Short,-1153.753134,-2935.548551,0.247686,308000.0,143.374,3.078566e+05
...,...,...,...,...,...,...,...,...,...
2023-07-13 14:41:00,Option.Call,4.0,LongShort.Short,-1081.569992,-2830.660069,0.211583,71176380.0,143.374,7.117624e+07
2023-07-13 14:42:00,Option.Call,4.0,LongShort.Short,-1069.267726,-2827.916473,0.211307,71313440.0,143.374,7.131330e+07
2023-07-13 14:43:00,Option.Call,4.0,LongShort.Short,-1076.680044,-2829.683443,0.210197,71448630.0,143.374,7.144849e+07
2023-07-13 14:44:00,Option.Call,4.0,LongShort.Short,-1084.997634,-2831.472205,0.207136,71583930.0,143.374,7.158379e+07


In [2]:
from Modules import Helpers

ModuleNotFoundError: No module named 'Modules'

In [45]:
for ticker in portfolio:
    vega = 0
    for token in ticker.tokens.values():
        x = token.stats['net_vega']
        vega += x[x != 0][0]
    print(ticker.symbol, vega)

BANKNIFTY -7552.050055029735
HDFCBANK 2459.3698892295
ICICIBANK 2640.9678984266097
KOTAKBANK 1383.3298841394526
SBIN 1142.7102516705954
AXISBANK 1008.2291627749024


C:\Users\vinayak\AppData\Local\Temp\20\ipykernel_90772\3222907251.py:5: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [42]:
import importlib
importlib.reload(Plot)

<module 'Modules.Plot' from 'C:\\Users\\vinayak\\Desktop\\Backtesting\\Modules\\Plot.py'>

In [33]:
visualise_dispersion_trade(
    start=index.df_futures.index[0], 
    end=index.df_futures.index[-1], 
    file_name='PLOT_Complete_Period_Summary.html'
)

In [34]:
save_dispersion_trades(
    start=index.df_futures.index[0], 
    end=index.df_futures.index[-1], 
    file_name=f'Info_Trades_for_{strategy_desc}.xlsx'
)

In [49]:
def metrics(fund_blocked):
    info_metrics = open(f'INFO_Metrics_for_{strategy_desc}.txt', 'w')
    sys.stdout = info_metrics

    print(f"TimeFrame Length: 6 months")
    print(f"Fundblocked in rs: {fund_blocked/100}")
    result_df.index = pd.to_datetime(result_df.index)
    result_df['date'] = result_df.index.date  # Create a new column for the date
    daily_profit = result_df.groupby('date')['running_pnl'].agg(lambda x: x.iloc[-1] - x.iloc[0])
    daily_profit.index = pd.to_datetime(result_df['date'].unique())
    
    ret = daily_profit.sum()/fund_blocked * 100
    print(f"Return: {round(ret)}%")
    
    sharpe_value = (daily_profit.mean() - risk_free_rate*fund_blocked/365)/daily_profit.std()
    print(f"Sharpe: {round(sharpe_value)}")
    
    peaks = result_df['running_pnl'].cummax()
    drawdowns = (result_df['running_pnl'] - peaks)/(peaks)
    max_drawdown = drawdowns.min()
    print(f"Drawdown: {round(max_drawdown)}%")
    monthly_data = daily_profit.resample('ME').sum()
    monthly_profit = monthly_data[monthly_data > 0].sum()
    monthly_loss = abs(monthly_data[monthly_data < 0].sum())
    if monthly_loss != 0:
        profit_loss_ratio = monthly_profit / monthly_loss
    else:
        profit_loss_ratio = float('inf')  # Avoid division by zero
    print("Monthly Profit/Loss Ratio:", round(profit_loss_ratio))
    
    windays = (result_df['running_pnl'] > 0).sum()
    lossdays = (result_df['running_pnl'] < 0).sum()
    if (windays + lossdays) > 0:
        win_ratio = windays / (windays + lossdays)
    else:
        win_ratio = 0
    print("Win Ratio:", round(win_ratio))
    
    streak = (result_df['running_pnl'] > 0).astype(int)  # 1 for winning days, 0 otherwise
    losing_streak = (result_df['running_pnl'] < 0).astype(int)  # 1 for losing days, 0 otherwise
    winning_streaks = streak.groupby((streak != streak.shift()).cumsum()).cumsum()
    losing_streaks = losing_streak.groupby((losing_streak != losing_streak.shift()).cumsum()).cumsum()
    print(f"Longest Winning Streak: {winning_streaks.max()}, Longest Losing Streak: {losing_streaks.max()}")

    sys.stdout = original_stdout.value
    info_metrics.close()